In [1]:
# 1. 필요한 패키지 설치
!pip install fastapi pyngrok uvicorn fsspec==2025.3.0 transformers datasets accelerate nest_asyncio --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 127.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 104.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [ ]:
# 2. 모델 로딩
from datasets import load_dataset, Dataset
from transformers import GPTNeoForCausalLM, GPT2Tokenizer, TrainingArguments, Trainer, DataCollatorForLanguageModeling
import torch, json

model_name = "EleutherAI/gpt-neo-1.3B"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPTNeoForCausalLM.from_pretrained(model_name).to("cuda" if torch.cuda.is_available() else "cpu")
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.31G [00:00<?, ?B/s]

In [ ]:
# 3. JSON 데이터 업로드
from google.colab import files
uploaded = files.upload()

with open("dummy_questions_100.json", "r", encoding="utf-8") as f:
    raw_data = json.load(f)

Saving dummy_questions_100.json to dummy_questions_100.json


In [ ]:
# 4. 'prompt', 'completion' 형식으로 변환
formatted_data = []
for item in raw_data:
    prompt = f"Question: {item['Question']}\n"
    completion = f"Answer: {item['Answer']}"
    formatted_data.append({"Text": prompt + completion})

In [ ]:
# 5. HuggingFace Dataset으로 변환
dataset = Dataset.from_list(formatted_data)

def tokenize(batch):
    return tokenizer(batch["Text"], padding="max_length", truncation=True, max_length=384)

tokenized_dataset = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/108 [00:00<?, ? examples/s]

In [ ]:
# 8. 데이터 콜레이터 설정
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
# 9. 학습 설정
training_args = TrainingArguments(
    output_dir="./gptneo-finetuned",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    evaluation_strategy="no",
    logging_steps=50,
    save_steps=200,
    save_total_limit=1,
    fp16=torch.cuda.is_available(),
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# 10. Trainer 구성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

<ipython-input-8-35355c832a15>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# 11. 학습 시작
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: circlehalf17 (circlehalf17-no-job) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
50,1.175900
100,0.516700
150,0.433000
200,0.356600
250,0.300000


TrainOutput(global_step=270, training_loss=0.5393146214661775, metrics={'train_runtime': 191.5857, 'train_samples_per_second': 2.819, 'train_steps_per_second': 1.409, 'total_flos': 1503511581818880.0, 'train_loss': 0.5393146214661775, 'epoch': 5.0})

In [ ]:
model.save_pretrained("/content/drive/MyDrive/gptneo-1.3B-finetuned")
tokenizer.save_pretrained("/content/drive/MyDrive/gptneo-1.3B-finetuned")

('/content/drive/MyDrive/gptneo-1.3B-finetuned/tokenizer_config.json',
 '/content/drive/MyDrive/gptneo-1.3B-finetuned/special_tokens_map.json',
 '/content/drive/MyDrive/gptneo-1.3B-finetuned/vocab.json',
 '/content/drive/MyDrive/gptneo-1.3B-finetuned/merges.txt',
 '/content/drive/MyDrive/gptneo-1.3B-finetuned/added_tokens.json')

In [2]:
# 2. 모델 로딩
from datasets import load_dataset, Dataset
from transformers import GPTNeoForCausalLM, GPT2Tokenizer, TrainingArguments, Trainer, DataCollatorForLanguageModeling
import torch, json

model_name = "/content/drive/MyDrive/gptneo-1.3B-finetuned"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPTNeoForCausalLM.from_pretrained(model_name).to("cuda" if torch.cuda.is_available() else "cpu")
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
# 4. 정규표현식 파싱 함수 정의
import re

def parse_problems_from_text(text):
    pattern = r"Question\s*[:：]\s*(.*?)\s*Answer\s*[:：]\s*(.*?)(?=\nQuestion|\Z)"
    matches = re.findall(pattern, text, re.DOTALL)
    return [{"Question": q.strip(), "Answer": a.strip()} for q, a in matches]

In [4]:
# 5. FastAPI app 설정
from fastapi import FastAPI
from pydantic import BaseModel
from transformers import pipeline

app = FastAPI()
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

class ProblemRequest(BaseModel):
#    question_type: str
    topic: str
    count: int
'''
class FeedbackRequest(BaseModel):
    question_id: str
    original_question: str
    updated_question: str
    feedback_type: str  # e.g. "incorrect_answer", "clarity", "typo"
    comment: str
'''
@app.post("/generate")
def generate_problems(req: ProblemRequest):
    prompt = f"""
    {req.topic}에 대한 {req.count}개의 문제를 생성하고 답을 함께 작성해줘.
    """

    result = generator(prompt, max_length=400, do_sample=True, temperature=0.8, top_p=0.9, repetition_penalty=1)
    output = result[0]["generated_text"]
    print("\nGPT Output:\n", output)

    try:
        problems = parse_problems_from_text(output)
        if not problems:
            raise ValueError("No questions could be parsed.")
    except Exception as e:
        problems = [{"Question": output if isinstance(output, str) else str(output), "Answer": f"⚠️ Parsing error: {str(e)}"}]

    return {
        "results": problems,
#        "raw_output": output
    }
'''
@app.post("/feedback")
def receive_feedback(feedback: FeedbackRequest):
    print("\n📥 Received Feedback:")
    print("Question ID:", feedback.question_id)
    print("Original:", feedback.original_question)
    print("Updated:", feedback.updated_question)
    print("Type:", feedback.feedback_type)
    print("Comment:", feedback.comment)

    # Here, you would typically store the feedback into a database or file for future model improvement.

    return {"status": "Feedback received", "question_id": feedback.question_id}
'''

Device set to use cpu


'\n@app.post("/feedback")\ndef receive_feedback(feedback: FeedbackRequest):\n    print("\n📥 Received Feedback:")\n    print("Question ID:", feedback.question_id)\n    print("Original:", feedback.original_question)\n    print("Updated:", feedback.updated_question)\n    print("Type:", feedback.feedback_type)\n    print("Comment:", feedback.comment)\n\n    # Here, you would typically store the feedback into a database or file for future model improvement.\n\n    return {"status": "Feedback received", "question_id": feedback.question_id}\n'

In [ ]:
# 6. Run ngrok + uvicorn server (Colab only)
from pyngrok import ngrok
import nest_asyncio
import uvicorn

ngrok.set_auth_token("2v1Fi5CEzLumREBpheNMIIepRlM_7uLFbq5PGe81hmEZiAe9K")  # Replace with your ngrok token
ngrok.kill()  # Kill previous tunnels

public_url = ngrok.connect(3000)
print("🔗 Public URL:", public_url.public_url)

nest_asyncio.apply()
uvicorn.run(app, host="0.0.0.0", port=3000)


INFO:     Started server process [839]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:3000 (Press CTRL+C to quit)


🔗 Public URL: https://f7e8-35-192-101-119.ngrok-free.app
INFO:     155.230.85.158:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     155.230.85.158:0 - "GET /openapi.json HTTP/1.1" 200 OK


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.



GPT Output:
 
    Math에 대한 0개의 문제를 생성하고 답을 함께 작성해줘.
    하나의 0이 없을 거래요.
    그래요.
    1 + 13 = 14.50.
    14.50가 더 복사장에서 0.50개의 기본이 없을 거래요.
    0.50개의 기본이 없을 거래요.
    0.50개의 기본이 없을 거래요.
    0.50개의 기본이 없을 거래요.
    0.50개의 기본이 없을 거래요.
    0.50개의 기본이 없을 거래요.
    0.50개의 기본이 없을 거래�
INFO:     155.230.85.158:0 - "POST /generate HTTP/1.1" 200 OK


In [ ]:
import time
while True:
    print("Still alive...")
    time.sleep(60)  # 1분마다 한번씩 출력

Still alive...
